In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_49.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_67_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_90.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_77.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_20_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_54.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_106_1.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_42.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_81.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment-2025/dataset/audios/test/audio_107.wav
/kaggle/input/shl-in

In [2]:
import os
import numpy as np
import pandas as pd

import librosa

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



In [3]:
DATASET_PATH = "/kaggle/input/shl-intern-hiring-assessment-2025/dataset"

TRAIN_AUDIO_PATH = os.path.join(DATASET_PATH, "audios/train")
TEST_AUDIO_PATH  = os.path.join(DATASET_PATH, "audios/test")

train_df = pd.read_csv(os.path.join(DATASET_PATH, "csvs/train.csv"))
test_df  = pd.read_csv(os.path.join(DATASET_PATH, "csvs/test.csv"))

print("Train samples:", len(train_df))
print("Test samples:", len(test_df))


Train samples: 409
Test samples: 197


In [4]:
def extract_features(audio_path):
    try:
        y, sr = librosa.load(audio_path, sr=16000)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        return np.mean(mfcc.T, axis=0)
    except Exception:
        return None




In [5]:
X = []
y = []
skipped = 0

for _, row in train_df.iterrows():
    filename = row["filename"] + ".wav"
    label = row["label"]

    path = os.path.join(TRAIN_AUDIO_PATH, filename)
    features = extract_features(path)

    if features is not None:
        X.append(features)
        y.append(label)
    else:
        skipped += 1

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Skipped files:", skipped)




/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/libro

X shape: (409, 40)
y shape: (409,)
Skipped files: 0


In [6]:
if len(X) > 0:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)

    val_preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, val_preds, squared=False)

    print("Validation RMSE:", rmse)
else:
    print("Training skipped due to unavailable audio files.")


Validation RMSE: 0.736237654664811


In [7]:
X_test = []
test_filenames = []

for _, row in test_df.iterrows():
    filename = row["filename"] + ".wav"
    path = os.path.join(TEST_AUDIO_PATH, filename)

    features = extract_features(path)

    if features is not None:
        X_test.append(features)
        test_filenames.append(row["filename"])

X_test = np.array(X_test)

print("Test features shape:", X_test.shape)



/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_13/3099497936.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.11/dist-packages/libro

Test features shape: (197, 40)


In [8]:
if len(X) > 0 and len(X_test) > 0:
    test_preds = model.predict(X_test)
else:
    # fallback: mean prediction (safe submission)
    test_preds = np.full(len(X_test), y.mean() if len(y) > 0 else 3.0)

submission = pd.DataFrame({
    "filename": test_filenames,
    "label": test_preds
})

submission.to_csv("submission.csv", index=False)
submission.head()



,filename,label
0,audio_141,3.5400
1,audio_114,2.6950
2,audio_17,2.9775
3,audio_76,2.8250
4,audio_156,2.7900
